Building Dense Vectors Using Transformers

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
# We will be using the sentence-transformers/stsb-distilbert-base model to build our dense vectors
# First we initialize our model and tokenizer:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-distilbert-base')
model = AutoModel.from_pretrained('sentence-transformers/stsb-distilbert-base')


In [3]:
# Then we tokenize a sentence just as we have been doing before:
text = "hello world what a time to be alive!"

tokens = tokenizer.encode_plus(text, max_length=128,
                               truncation=True, padding='max_length',
                               return_tensors='pt')

In [4]:
# We process these tokens through our model:
outputs = model(**tokens)
outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.9489,  0.6905, -0.2188,  ...,  0.0161,  0.5874, -0.1449],
         [-0.6643,  1.1984, -0.1346,  ...,  0.4839,  0.6338, -0.5003],
         [-0.3289,  0.6412,  0.2473,  ..., -0.0965,  0.4298,  0.0515],
         ...,
         [-0.7853,  0.8094, -0.2639,  ...,  0.2177,  0.3335,  0.1107],
         [-0.7528,  0.6285, -0.0088,  ...,  0.1024,  0.4585,  0.1720],
         [-1.0754,  0.4878, -0.3458,  ...,  0.2764,  0.5604,  0.1236]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

The dense vector representations of our text are contained within the outputs 'last_hidden_state' tensor, which we access like so:

In [5]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[-0.9489,  0.6905, -0.2188,  ...,  0.0161,  0.5874, -0.1449],
         [-0.6643,  1.1984, -0.1346,  ...,  0.4839,  0.6338, -0.5003],
         [-0.3289,  0.6412,  0.2473,  ..., -0.0965,  0.4298,  0.0515],
         ...,
         [-0.7853,  0.8094, -0.2639,  ...,  0.2177,  0.3335,  0.1107],
         [-0.7528,  0.6285, -0.0088,  ...,  0.1024,  0.4585,  0.1720],
         [-1.0754,  0.4878, -0.3458,  ...,  0.2764,  0.5604,  0.1236]]],
       grad_fn=<NativeLayerNormBackward0>)

In [7]:
# 128 because max_length = 128
embeddings.shape

torch.Size([1, 128, 768])

After we have produced our dense vectors embeddings, we need to perform a mean pooling operation on them to create a single vector encoding (the sentence embedding). To do this mean pooling operation we will need to multiply each value in our embeddings tensor by it's respective attention_mask value - so that we ignore non-real tokens.

To perform this operation, we first resize our attention_mask tensor:

In [11]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([1, 128])

In [9]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([1, 128, 768])

In [12]:
mask[0][0].shape

torch.Size([768])

In [13]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

Each vector above represents a single token attention mask - each token now has a vector of size 768 representing it's attention_mask status. Then we multiply the two tensors to apply the attention mask:

In [14]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([1, 128, 768])

In [15]:
masked_embeddings

tensor([[[-0.9489,  0.6905, -0.2188,  ...,  0.0161,  0.5874, -0.1449],
         [-0.6643,  1.1984, -0.1346,  ...,  0.4839,  0.6338, -0.5003],
         [-0.3289,  0.6412,  0.2473,  ..., -0.0965,  0.4298,  0.0515],
         ...,
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<MulBackward0>)

Then we sum the remained of the embeddings along axis 1:

In [16]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([1, 768])

Then sum the number of values that must be given attention in each position of the tensor:

In [18]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([1, 768])

In [19]:
summed_mask

tensor([[11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11., 11.,
         11., 11., 11., 11., 11., 11., 11., 11., 11.

Finally, we calculate the mean as the sum of the embedding activations summed divided by the number of values that should be given attention in each position summed_mask:

In [20]:
mean_pooled = summed / summed_mask
mean_pooled

tensor([[-3.8485e-01,  7.8107e-01, -1.7720e-01, -1.4125e+00, -2.3358e-01,
          9.0891e-01, -7.8390e-02,  6.0347e-01,  6.7886e-02, -3.9841e-01,
          3.9223e-02, -4.6774e-01, -7.1848e-01, -1.1863e-01, -7.1194e-02,
          6.6018e-03, -1.4093e-01,  3.1271e-01, -6.5574e-01, -1.6470e-01,
         -1.0026e-01, -3.8357e-01,  6.1278e-02, -7.3818e-01, -5.9918e-01,
          2.8855e-01,  8.6372e-01,  5.8388e-01, -3.5059e-02,  4.3197e-01,
         -5.0111e-01, -4.3498e-01,  2.3498e-01, -3.7127e-01, -1.0044e+00,
          1.0000e+00, -2.1000e+00, -3.2251e-01, -1.6085e-01, -7.3701e-01,
          5.4928e-01, -1.2066e-01,  7.2698e-01, -5.0328e-02, -1.7545e+00,
          8.0573e-01, -5.0553e-01, -4.7172e-01, -1.6727e-01,  5.9727e-01,
          5.6203e-01, -3.6104e-01, -1.6429e-01, -5.5215e-01, -5.0418e-01,
          5.6187e-01, -1.1415e+00,  1.0771e+00,  5.5689e-01, -7.0632e-02,
         -2.6932e-01, -6.8905e-01,  1.8093e-01,  3.1045e-01,  3.9037e-02,
          3.1064e-01, -4.4495e-01, -4.

And that is how we calculate our dense similarity vector.

### Calculating Similarity
When calculating similarity between our transformer embedded vectors, we can use any of the three similarity metrics already covered.

But first, let's create some embeddings.

In [1]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "Standing on one's head at job interviews forms a lasting impression.",
    "It took him a month to finish the meal.",
    "He found a leprechaun in his walnut shell."
]

# thanks to https://randomwordgenerator.com/sentence.php

In [22]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# initialize dictionary that will contain tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # tokenize sentence and append to dictionary lists
    new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,
                                       padding='max_length', return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [23]:
tokens['input_ids'].shape

torch.Size([6, 128])

In [24]:
# We process these tokens through our model:

outputs = model(**tokens)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [25]:
# The dense vector representations of our text are contained within the outputs 'last_hidden_state' tensor, which we access like so:

embeddings = outputs.last_hidden_state
embeddings

tensor([[[-6.9229e-02,  6.2300e-01,  3.5370e-02,  ...,  8.0334e-01,
           1.6314e+00,  3.2812e-01],
         [ 3.6729e-02,  6.8419e-01,  1.9460e-01,  ...,  8.4759e-02,
           1.4747e+00, -3.0080e-01],
         [-1.2142e-02,  6.5431e-01, -7.2717e-02,  ..., -3.2600e-02,
           1.7717e+00, -6.8121e-01],
         ...,
         [ 1.9532e-01,  1.1085e+00,  3.3905e-01,  ...,  1.2826e+00,
           1.0114e+00, -7.2754e-02],
         [ 9.0217e-02,  1.0288e+00,  3.2973e-01,  ...,  1.2940e+00,
           9.8650e-01, -1.1125e-01],
         [ 1.2404e-01,  9.7365e-01,  3.9329e-01,  ...,  1.1359e+00,
           8.7685e-01, -1.0435e-01]],

        [[-3.2124e-01,  8.2512e-01,  1.0554e+00,  ..., -1.8555e-01,
           1.5169e-01,  3.9366e-01],
         [-7.1457e-01,  1.0297e+00,  1.1217e+00,  ...,  3.3118e-02,
           2.3820e-01, -1.5632e-01],
         [-2.3522e-01,  1.1353e+00,  8.5941e-01,  ..., -4.3096e-01,
          -2.7242e-02, -2.9677e-01],
         ...,
         [-5.4000e-01,  3

In [26]:
embeddings.shape

torch.Size([6, 128, 768])

After we have produced our dense vectors embeddings, we need to perform a mean pooling operation on them to create a single vector encoding (the sentence embedding). To do this mean pooling operation we will need to multiply each value in our embeddings tensor by it's respective attention_mask value - so that we ignore non-real tokens.

To perform this operation, we first resize our attention_mask tensor:

In [27]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([6, 128])

In [28]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([6, 128, 768])

In [29]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

Each vector above represents a single token attention mask - each token now has a vector of size 768 representing it's attention_mask status. Then we multiply the two tensors to apply the attention mask:

In [30]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([6, 128, 768])

In [31]:
masked_embeddings

tensor([[[-0.0692,  0.6230,  0.0354,  ...,  0.8033,  1.6314,  0.3281],
         [ 0.0367,  0.6842,  0.1946,  ...,  0.0848,  1.4747, -0.3008],
         [-0.0121,  0.6543, -0.0727,  ..., -0.0326,  1.7717, -0.6812],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000]],

        [[-0.3212,  0.8251,  1.0554,  ..., -0.1855,  0.1517,  0.3937],
         [-0.7146,  1.0297,  1.1217,  ...,  0.0331,  0.2382, -0.1563],
         [-0.2352,  1.1353,  0.8594,  ..., -0.4310, -0.0272, -0.2968],
         ...,
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000,  0.0000]],

        [[-0.7576,  0.8399, -0.3792,  ...,  0.1271,  1.2514,  0.1365],
         [-0.6591,  0.7613, -0.4662,  ...,  0

In [32]:
# Then we sum the remained of the embeddings along axis 1:

summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([6, 768])

In [33]:
# Then sum the number of values that must be given attention in each position of the tensor:

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([6, 768])

In [34]:
summed_mask

tensor([[15., 15., 15.,  ..., 15., 15., 15.],
        [22., 22., 22.,  ..., 22., 22., 22.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [16., 16., 16.,  ..., 16., 16., 16.],
        [12., 12., 12.,  ..., 12., 12., 12.],
        [14., 14., 14.,  ..., 14., 14., 14.]])

In [35]:
# Finally, we calculate the mean as the sum of the embedding activations summed divided by the number of values that should be given attention in each position summed_mask:

mean_pooled = summed / summed_mask

In [36]:
mean_pooled

tensor([[ 0.0745,  0.8637,  0.1795,  ...,  0.7734,  1.7247, -0.1803],
        [-0.3715,  0.9729,  1.0840,  ..., -0.2552, -0.2759,  0.0358],
        [-0.5030,  0.7950, -0.1240,  ...,  0.1441,  0.9704, -0.1791],
        [-0.0132,  0.9773,  1.4516,  ..., -0.8462, -1.4004, -0.4118],
        [-0.2019,  0.0597,  0.8603,  ..., -0.0100,  0.8431, -0.0841],
        [-0.2131,  1.0175, -0.8833,  ...,  0.7371,  0.1947, -0.3011]],
       grad_fn=<DivBackward0>)

In [37]:
# And that is how we calculate our dense similarity vector.

from sklearn.metrics.pairwise import cosine_similarity

In [38]:
# Let's calculate cosine similarity for sentence 0:

# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.33088917, 0.72192585, 0.17475507, 0.44709665, 0.55483645]],
      dtype=float32)

So, as intended, the most similar sentence is that in index 2 - which contains the same meaning as our first sentence, without using the same words

We've worked through creating our embeddings using the transformers library - and at times it can be quite involved. Now, it's important to understand the steps, but we can make life easier by using the sentence-transformers library.

We'll work through the same process - but using sentence-transformers instead.

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
# Encode the sentences:

sentence_embeddings = model.encode(sentences)

In [4]:
sentence_embeddings.shape

(6, 768)

In [5]:
# And no we have our sentence embeddings - a much quicker approach. We then compare just as we did before using cosine similarity:

from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Let's calculate cosine similarity for sentence 0:

cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

array([[0.33088908, 0.72192574, 0.17475477, 0.44709638, 0.5548362 ]],
      dtype=float32)